# imports

In [ ]:
# beomi/kcbert-base
!pip install transformers
!pip install torchmetrics
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/QIA-Hackathon2023")

import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 33.7 MB/s eta 0:00:00
Mounted at /content/drive


# create dataset

In [ ]:

train=pd.read_csv('df_train_raw.csv', index_col=0)
test=pd.read_csv('df_valid_raw.csv', index_col=0)
df_questions = pd.read_excel("Question.xlsx")

df_submission_test = pd.read_csv("hackathon_test_for_user.csv", encoding="cp949")

In [ ]:
# MBTI mapping 
MBTI_mapping = [
    {'I': 0, 'E': 1},
    {'S': 0, 'N': 1},
    {'T': 0, 'F': 1},
    {'J': 0, 'P': 1},
]
# 0 - ISTJ; 1 - ENFP 

In [ ]:
def create_promt (row):
    #p = f"젠더: {row.Gender}.\n 나이: {row.Age}.\n 질문: {df_questions.Question.loc[row.Q_number-1]}.\n 정답: {row.Answer}.\n"
    p = f"질문: {df_questions.Question.loc[row.Q_number-1]}.\n 정답: {row.Answer}.\n"
    return p



In [ ]:
train["prompt"] = train.apply(create_promt, axis=1)
test["prompt"] = test.apply(create_promt, axis=1)
df_submission_test["prompt"] = df_submission_test.apply(create_promt, axis=1)


In [ ]:
def create_label_mbti (mbti):
  return np.array([MBTI_mapping[i][letter] for i, letter in enumerate(mbti)])

print ("INFP: ", create_label_mbti("INFP"))

def create_label_user_ID (user_ID):
  b = np.zeros([240])
  b[user_ID-1]=1
  return b

# test["label_mbti"] = test.MBTI.map( create_label_mbti)
# train["label_mbti"] = train.MBTI.map( create_label_mbti )

# train["label_userID"] = train.User_ID.map( create_label_user_ID)
# test["label_userID"] = test.User_ID.map( create_label_user_ID)

INFP:  [0 1 1 1]


In [ ]:
# IMPORTANT: ID's the model will predict are from 0 to 239
def create_label (row):
  #mbti = create_label_mbti(row.MBTI)
  id = row.User_ID-1 
  return np.array(id)


In [ ]:
train["label"] = train.apply(create_label, axis=1)
test["label"] = test.apply(create_label, axis=1)



In [ ]:
train.tail()

,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer,prompt,label
11507,14376,240,0,40,ISTJ,36,<아니다> 저는 즐거운 파티나 행사로 일주일 피로를 푸는 편이 아닙니다. 이유는 그...,질문: 혼자만의 시간을 보내기보다는 즐거운 파티와 행사로 일주일의 피로를 푸는 편인...,239
11508,14377,240,0,40,ISTJ,37,<중립> 저는 미술관 가는 일을 좋아하지 않습니다. 이유는 미술 작품에 관심이 많이...,질문: 미술관에 가는 일을 좋아하나요? 꼭 미술관이 아니라도 예술작품을 감상하는 일...,239
11509,14378,240,0,40,ISTJ,38,<그렇다> 저는 다른 사람의 감정을 이해하기 힘들 때가 많습니다. 이유는 각자의 살...,질문: 다른 사람의 감정을 이해하기 힘들 때가 많나요? 어떤 상황에서 그런지 예를 ...,239
11510,14379,240,0,40,ISTJ,39,<아니다> 저는 매일 할 일을 계획하지 않습니다. 이유는 매일매일보다는 크게 한 건...,질문: 매일 할 일을 계획하는 일을 좋아하나요? 업무 또는 개인적인 일 등을 포함해...,239
11511,14380,240,0,40,ISTJ,40,<아니다> 저는 불안함을 많이 느낍니다. 이유는 모든 것이 확실히 되야지만 마음이 ...,질문: 불안함을 느낄 때가 거의 없나요? 타고난 성향인지 아니면 그렇게 된 어떤 계...,239


# def model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

class KoreanTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long),
        }


In [ ]:
from torch.nn import functional as F
from transformers import AutoModel
# model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=2)

class MBTI_UserID_Model(torch.nn.Module):

    def __init__(self, large = False, dropout_per = 0.1, total_Users = 240):
        super(MBTI_UserID_Model, self).__init__()        
        
        if large: 
          model_features = 1024
          self.base_model = AutoModel.from_pretrained("beomi/kcbert-large", )
        
        else: 
          model_features = 768
          self.base_model = AutoModel.from_pretrained("beomi/kcbert-base", )

        #self.linear1 = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, 1))
        #self.linear2 = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, 1))
        #self.linear3 = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, 1))
        #self.linear4 = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, 1))
        self.linear_users = torch.nn.Sequential(torch.nn.Dropout(p=dropout_per), torch.nn.Linear(model_features, total_Users))
        

    def forward(self, x, attention_mask):
        x = self.base_model (x, attention_mask=attention_mask).pooler_output

        return self.linear_users(x)#self.linear1(x), self.linear2(x), self.linear3(x), self.linear4(x), 



# set up training

In [ ]:
test.head()

,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer,prompt,label
40,41,1,1,30,INFP,41,<그렇다> 말을 잘 못하기 때문에 말실수를 할까 봐 피하고 싶습니다. 문자로 하는 ...,질문: 전화 통화를 거는 일은 가능한 피하고 싶나요? 이유는 무엇인가요..\n 정답...,0
41,42,1,1,30,INFP,42,<그렇다> 아주 많은 시간을 할애하는 편은 아니지만 노력하는 편입니다. 너무 다른 ...,질문: 자신의 의견과 매우 다른 의견을 이해하기 위해 많은 시간을 할애하곤 하나요?...,0
42,43,1,1,30,INFP,43,<아니다> 대부분 친구가 먼저 만나자고 하면 가는 편입니다. 저는 연락을 대체로 먼...,질문: 친구에게 먼저 만나자고 연락하는 편인가요? 어떤가요..\n 정답: <아니다>...,0
43,44,1,1,30,INFP,44,<그렇다> 차질이 생기면 그 부분만 다르게 하고 원래 계획으로 돌아가기 위해 노력합...,질문: 계획에 차질이 생기면 최대한 빨리 계획으로 돌아가기 위해 노력하나요? 최근에...,0
44,45,1,1,30,INFP,45,<그렇다> 굉장히 오래전의 실수라 이젠 바뀔 수 없음에도 종종 후회합니다. 수능 끝...,질문: 오래전의 실수를 후회할 때가 많나요? 요즘에 무엇때문에 그런가요..\n 정답...,0


In [ ]:
use_train_val_combined = True
train_val_combined = pd.concat([train, test])
train_val_combined.tail()

,Data_ID,User_ID,Gender,Age,MBTI,Q_number,Answer,prompt,label
11515,14384,240,0,40,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...,질문: 계획에 차질이 생기면 최대한 빨리 계획으로 돌아가기 위해 노력하나요? 최근에...,239
11516,14385,240,0,40,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...,질문: 오래전의 실수를 후회할 때가 많나요? 요즘에 무엇때문에 그런가요..\n 정답...,239
11517,14386,240,0,40,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...,질문: 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않는 편인가요? 답변에 대한 ...,239
11518,14387,240,0,40,ISTJ,47,<아니다> 저는 감정에 휘둘리는 편이 아닙니다. 이유는 감정을 감추고 밖으로 표현하...,질문: 감정을 조절하기보다는 감정에 휘둘리는 편인가요? 본인의 생각과 주변의 평가는...,239
11519,14388,240,0,40,ISTJ,48,<아니다> 저는 상대방 잘못일 때 상대방의 체면을 살려주기 위해 노력하지 않습니다....,질문: 상대방의 잘못이라는 것이 명백할 때도 상대방의 체면을 살려주기 위해 노력하나...,239


In [ ]:
train_model_type_large = False
train_model_type_large = True

if train_model_type_large: 
  tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
else: 
  tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

if use_train_val_combined: 
  train_texts=list(train_val_combined['prompt'])
  train_labels= list(train_val_combined['label'])
else: 
  train_texts=list(train['prompt'])
  train_labels= list(train['label'])
# train_labels= [list(train['label_mbti']), list(train['label_userID'])]

test_texts=list(test['prompt'])
test_labels= list(test['label'])
# test_labels= [list(test['label_mbti']), list(test['label_userID'])]

batch_size = 32

max_length = 128

train_dataset = KoreanTextDataset(train_texts, train_labels, tokenizer, max_length=max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = KoreanTextDataset(test_texts, test_labels, tokenizer, max_length=max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [ ]:
# model = MBTI_UserID_Model(large = train_model_type_large, dropout_per = 0.3 )

model = MBTI_UserID_Model(large = train_model_type_large, dropout_per = 0.5 )

Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# model eval

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print (device)

cuda


In [ ]:
from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import roc_auc_score, accuracy_score

loss_cross_en_userID = torch.nn.CrossEntropyLoss()
user_id_loss_coef = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# eval loop:
def model_eval(): 
  model.eval()
  #predictions = [[] for i in range(4)]
  #true_labels = [[] for i in range(4)]

  predictions_usrID = []
  true_labels_usrID = []

  with torch.no_grad():
      total_human_loss = 0
      total_mbti_loss = 0
      iter = 0
      for batch in test_loader:

          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          outputs = model(input_ids, attention_mask=attention_mask)
          labels = batch['label'].to(device)

          
          loss = 0
          #for i in range(4): 
              #loss += F.binary_cross_entropy_with_logits( outputs[i][:, 0], labels[:,i].type(torch.FloatTensor).to(device) ) 
          
          # user ID class loss: 
          humna_class_loss = user_id_loss_coef*loss_cross_en_userID(outputs, labels)
          
          total_mbti_loss+=loss
          total_human_loss+=humna_class_loss
          
          iter += 1

          # labels.cpu().numpy()
          # for i in range(4):
            
          #   logits = outputs[i]
          #   prob = F.sigmoid(logits).cpu().numpy() 
          #   predictions[i].append(prob)
            
          #   label = batch['label'][:, i]
          #   true_labels[i].append(label)
          
          prob = outputs.softmax(dim=1).cpu().numpy() 
          predictions_usrID.append(prob)
          label = batch['label']#[:, 4]
          true_labels_usrID.append(label)

  #for i in range(4):
      #score = roc_auc_score(np.concatenate(true_labels[i]), np.concatenate(predictions[i]))
      #print (f"score for {i}: {score}. ")
  accuracy_score_user = accuracy_score (  np.concatenate(true_labels_usrID), np.concatenate(predictions_usrID).argmax(axis = 1) )

  total_human_loss = total_human_loss/iter
  #total_mbti_loss = total_mbti_loss/iter

  print ("accuracy:", accuracy_score_user)
  print ("losses:", float(total_human_loss.cpu().numpy()))


# training loop

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-6)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
lr = 1e-5
# lr = 2e-6

epochs = 15
loss_cross_en_userID = torch.nn.CrossEntropyLoss()
user_id_loss_coef = 1

from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import roc_auc_score

def sigmoid (x):
  return 1/(1 + np.exp ( -x ))


In [ ]:
epochs = 13

model_name = "large_3_"
# Training loop
for epoch in range(epochs):
    if epoch == 0: 
      optimizer = torch.optim.Adam(model.parameters(), lr=2e-6)
    else: 
      optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    total_loss = 0

    total_human_loss = 0
    #total_mbti_loss = 0

    #predictions = [[] for i in range(4)]
    #true_labels = [[] for i in range(4)]

    predictions_usrID = []
    true_labels_usrID = []


    iter = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, )

        loss = 0
        #for i in range(4): 
            #loss += F.binary_cross_entropy_with_logits( outputs[i][:, 0].to(device), labels[:,i].type(torch.FloatTensor).to(device)) 

        #total_mbti_loss+=loss
        # user ID class loss: 
        humna_class_loss = user_id_loss_coef*loss_cross_en_userID(outputs, labels)
        total_human_loss+=humna_class_loss
        loss+= humna_class_loss

        loss.backward()
        optimizer.step()
        total_loss += loss
        iter += 1


        # labels.cpu().numpy()
        #for i in range(4):
            
            #logits = outputs[i]
            #prob = F.sigmoid(logits).detach().cpu().numpy() 
            #predictions[i].append(prob)
            
            #label = batch['label'][:, i]
            #true_labels[i].append(label)
          
        prob = outputs.softmax(dim=1).detach().cpu().numpy() 
        predictions_usrID.append(prob)
        label = batch['label']
        true_labels_usrID.append(label)
        
    
    print()
    print (f"Train results epoh {epoch}:")
    #for i in range(4):
      #score = roc_auc_score(np.concatenate(true_labels[i]), np.concatenate(predictions[i]))
      #print (f"score for {i}: {score}. ")
    accuracy_score_user = accuracy_score (  np.concatenate(true_labels_usrID), np.concatenate(predictions_usrID).argmax(axis = 1) )
    print (f"accuracy_train: {accuracy_score_user}. ")

    
    print(f"Done epoch: {epoch}. Train loss = {total_loss/iter}")
    total_human_loss = total_human_loss/iter
    #total_mbti_loss = total_mbti_loss/iter

    print ("losses:", float(total_human_loss.detach().cpu().numpy()))

    print()
    print (f"Eval results at epoh {epoch}: ")
    model_eval()



Train results epoh 0:
accuracy_train: 0.005208333333333333. 
Done epoch: 0. Train loss = 5.578374862670898
losses: 5.578374862670898

Eval results at epoh 0: 
accuracy: 0.007291666666666667
losses: 5.470431804656982

Train results epoh 1:
accuracy_train: 0.007725694444444445. 
Done epoch: 1. Train loss = 5.437248706817627
losses: 5.437248706817627

Eval results at epoh 1: 
accuracy: 0.0203125
losses: 5.179555416107178

Train results epoh 2:
accuracy_train: 0.018229166666666668. 
Done epoch: 2. Train loss = 5.1848554611206055
losses: 5.1848554611206055

Eval results at epoh 2: 
accuracy: 0.04010416666666667
losses: 4.841357707977295

Train results epoh 3:
accuracy_train: 0.03428819444444445. 
Done epoch: 3. Train loss = 4.835360050201416
losses: 4.835360050201416

Eval results at epoh 3: 
accuracy: 0.0703125
losses: 4.542413711547852

Train results epoh 4:
accuracy_train: 0.05555555555555555. 
Done epoch: 4. Train loss = 4.587497234344482
losses: 4.587497234344482

Eval results at epoh

In [ ]:
PATH = "your path"
torch.save(model.state_dict(), PATH)


# create submission file

In [ ]:
sub_file_prefix = "small_2"
sub_file_prefix = model_name + "_add_"
print (sub_file_prefix)
"hackathon_test_for_user.csv"
df_submission_test


large_3__add_


,Gender,Age,Q_number,Answer,prompt
0,0,30,59,<아니다> 저는 모든 일은 정해진 시간을 지켜서 해야 된다고 생각되어서 마감 기한을...,질문: 마감 기한을 지키기가 힘든가요? 경험을 이야기해보아요..\n 정답: <아니다...
1,1,40,53,<중립> 저는 조용하고 사적인 장소도 좋아하고 사람들로 붐비고 떠들썩한 장소도 좋아...,질문: 조용하고 사적인 장소보다는 사람들로 붐비고 떠들썩한 장소를 좋아하나요? 답변...
2,1,40,56,<그렇다> 저는 규칙을 잘 지키고 매뉴얼 대로 일하는 사람입니다. 그래서 데이터 라...,질문: 단계를 건너뛰는 일 없이 절차대로 일을 완수하는 편인가요? 그러한 최근 경험...
3,1,40,60,<그렇다> 저는 항상 긍정적인 사고방식을 가지고 살려고 노력하고 있습니다. 이유는 ...,질문: 일이 원하는 대로 진행될 것이라는 자신감이 있나요? 그렇게 된 계기나 이유가...
4,1,30,51,<중립> 혼자서 일하는 것도 좋고 함께 일하는 것도 모두 좋은데 같이 의논하는 일도...,질문: 대부분의 시간을 혼자서 일할 수 있는 직업을 원하나요? 이유도 말씀해주세요....
...,...,...,...,...,...
2875,1,20,59,<아니다> 마감 기한은 웬만해서는 지키려고 하는 편이에요. 거의 대부분 지각하지 않아요.,질문: 마감 기한을 지키기가 힘든가요? 경험을 이야기해보아요..\n 정답: <아니다...
2876,0,20,54,<중립> 저는 상대방의 감정을 바로는 아니지만 어느 정도 알아차릴 수는 있습니다. ...,질문: 상대방의 감정을 바로 알아차릴 수 있나요? 그러한 사례가 많나요..\n 정답...
2877,1,30,50,<그렇다> 상대방이 높게 평가할 수 롤 작은 실수도 크게 보일 수 있기 때문이에요....,질문: 상대방이 자신을 높게 평가하면 나중에 상대방이 실망하게 될까 걱정하곤 하나요...
2878,1,30,50,<아니다> 상대방이 나를 높게 평가하면 더 잘하면 된다고 생각하기 때문에 상대방이 ...,질문: 상대방이 자신을 높게 평가하면 나중에 상대방이 실망하게 될까 걱정하곤 하나요...


In [ ]:
df_submission_test
batch_size = 200

test_texts_submission=list(df_submission_test['prompt'])
test_labels_submission=list(df_submission_test['Q_number'])

if train_model_type_large: 
  tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
else: 
  tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

test_dataset_sub = KoreanTextDataset(test_texts_submission, test_labels_submission, tokenizer, max_length=max_length)
test_loader_sub = DataLoader(test_dataset_sub, batch_size=batch_size, shuffle=False)


In [ ]:
model.eval()
model.to(device)
predictions = [[] for i in range(4)]
true_labels = [[] for i in range(4)]

predictions_usrID = []


with torch.no_grad():

    for batch in test_loader_sub:

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)

        # userID predictions
        prob = outputs.softmax(dim=1).cpu().numpy() 
        predictions_usrID.append(prob)

        # labels.cpu().numpy()
        #for i in range(4):
          
          #logits = outputs[i]
          #prob = F.sigmoid( logits).cpu().numpy()
          #predictions[i].append(prob)
          

print(f"done epoch:  ")


done epoch:  


In [ ]:
ids_predictions = np.concatenate(predictions_usrID)
ids_predictions.shape

(2880, 240)

In [ ]:
df_userID_map = train.iloc[::40,][["User_ID", "label", "MBTI"]]
df_userID_map.index = df_userID_map.User_ID
df_userID_dict = dict(df_userID_map["label"])

In [ ]:
df_submission_test

,Gender,Age,Q_number,Answer,prompt
0,0,30,59,<아니다> 저는 모든 일은 정해진 시간을 지켜서 해야 된다고 생각되어서 마감 기한을...,질문: 마감 기한을 지키기가 힘든가요? 경험을 이야기해보아요..\n 정답: <아니다...
1,1,40,53,<중립> 저는 조용하고 사적인 장소도 좋아하고 사람들로 붐비고 떠들썩한 장소도 좋아...,질문: 조용하고 사적인 장소보다는 사람들로 붐비고 떠들썩한 장소를 좋아하나요? 답변...
2,1,40,56,<그렇다> 저는 규칙을 잘 지키고 매뉴얼 대로 일하는 사람입니다. 그래서 데이터 라...,질문: 단계를 건너뛰는 일 없이 절차대로 일을 완수하는 편인가요? 그러한 최근 경험...
3,1,40,60,<그렇다> 저는 항상 긍정적인 사고방식을 가지고 살려고 노력하고 있습니다. 이유는 ...,질문: 일이 원하는 대로 진행될 것이라는 자신감이 있나요? 그렇게 된 계기나 이유가...
4,1,30,51,<중립> 혼자서 일하는 것도 좋고 함께 일하는 것도 모두 좋은데 같이 의논하는 일도...,질문: 대부분의 시간을 혼자서 일할 수 있는 직업을 원하나요? 이유도 말씀해주세요....
...,...,...,...,...,...
2875,1,20,59,<아니다> 마감 기한은 웬만해서는 지키려고 하는 편이에요. 거의 대부분 지각하지 않아요.,질문: 마감 기한을 지키기가 힘든가요? 경험을 이야기해보아요..\n 정답: <아니다...
2876,0,20,54,<중립> 저는 상대방의 감정을 바로는 아니지만 어느 정도 알아차릴 수는 있습니다. ...,질문: 상대방의 감정을 바로 알아차릴 수 있나요? 그러한 사례가 많나요..\n 정답...
2877,1,30,50,<그렇다> 상대방이 높게 평가할 수 롤 작은 실수도 크게 보일 수 있기 때문이에요....,질문: 상대방이 자신을 높게 평가하면 나중에 상대방이 실망하게 될까 걱정하곤 하나요...
2878,1,30,50,<아니다> 상대방이 나를 높게 평가하면 더 잘하면 된다고 생각하기 때문에 상대방이 ...,질문: 상대방이 자신을 높게 평가하면 나중에 상대방이 실망하게 될까 걱정하곤 하나요...


In [ ]:
df_submission_test["userIDs predictions"] = [g for g in ids_predictions]

In [ ]:

df_submission_test.to_pickle("path.pkl")